In [66]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [112]:
journal_filename = "journals/shot.txt"

EVENT_TYPE = enums.Entity.Type.EVENT
PERSON_TYPE = enums.Entity.Type.PERSON
LOCATION_TYPE = enums.Entity.Type.LOCATION
OTHER_TYPE = enums.Entity.Type.OTHER

ALLOWABLE_LABELS = [enums.DependencyEdge.Label.DOBJ,
                    #enums.DependencyEdge.Label.NSUBJ,
                    #enums.DependencyEdge.Label.NSUBJPASS,
                    enums.DependencyEdge.Label.IOBJ]

In [113]:
client = language.LanguageServiceClient()

with open(journal_filename, 'r') as journal_file:
    # Instantiates a plain text document.
    content = journal_file.read()

document = types.Document(
    content=content,
    type=enums.Document.Type.PLAIN_TEXT)
annotations = client.analyze_entity_sentiment(document=document, encoding_type="UTF8")
syntax = client.analyze_syntax(document=document, encoding_type="UTF8")

In [114]:
def sm_fun(score, magnitude):
    return score * magnitude

In [115]:
offset_label_map = {tok.text.begin_offset: tok.dependency_edge.label for tok in syntax.tokens}

In [116]:
events = []
people = []
locations = []
other = []

for keyword in annotations.entities:
    word_type = keyword.type
    entity_obj = (keyword.name, sm_fun(keyword.sentiment.score, keyword.sentiment.magnitude))
    
    if keyword.type == EVENT_TYPE:
        events.append(entity_obj)
    
    elif keyword.type == PERSON_TYPE:
        people.append(entity_obj)
    
    elif keyword.type == LOCATION_TYPE:
        locations.append(entity_obj)
    
    elif keyword.type == OTHER_TYPE:
        allowable = False
        
        for mention in keyword.mentions:
            content = mention.text.content
            offset = mention.text.begin_offset
            
            for word in content.split(" "):
                try:
                    if offset_label_map[offset] in ALLOWABLE_LABELS:
                        allowable = True
                except:
                    print("Missed word somehow")
                offset += (len(word) + 1)
        
        if allowable:
            other.append(entity_obj)

In [117]:
events.sort(key=lambda x: -x[1])
people.sort(key=lambda x: -x[1])
locations.sort(key=lambda x: -x[1])
other.sort(key=lambda x: -x[1])

In [118]:
events

[]

In [119]:
people

[]

In [120]:
locations

[('mall', 0.0)]

In [121]:
other

[]

In [122]:
annotations.entities

[name: "mall"
type: LOCATION
salience: 1.0
mentions {
  text {
    content: "mall"
    begin_offset: 18
  }
  type: COMMON
  sentiment {
  }
}
sentiment {
}
]

In [78]:
syntax

sentences {
  text {
    content: "dear diary,"
  }
}
sentences {
  text {
    content: "today i accidentailly killed someone."
    begin_offset: 13
  }
}
sentences {
  text {
    content: "i ran them over with my car."
    begin_offset: 51
  }
}
sentences {
  text {
    content: "the weird thing is, I didn\'t feel bad."
    begin_offset: 81
  }
}
sentences {
  text {
    content: "I didn\'t know the person, so i didn\'t really care if they lived or died."
    begin_offset: 120
  }
}
sentences {
  text {
    content: "In fact, I think it was fun."
    begin_offset: 194
  }
}
sentences {
  text {
    content: "There\'s a strange power to being able to take some random person\'s life whenever you want."
    begin_offset: 223
  }
}
sentences {
  text {
    content: "All my life, I have felt powerless to my tormentors, and now, I have found a way to take back control."
    begin_offset: 314
  }
}
sentences {
  text {
    content: "anyways, talk to you soon; maybe the next time I write you,